This exercise will require you to pull some data from the Quandl API. Quandl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'API_KEY'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [3]:
# First, import the relevant modules
import requests
import seaborn as sns

In [15]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = "https://www.quandl.com/api/v3/datasets/WIKI/FB/data.json?api_key="+API_KEY
quandl = requests.get(url)
quandl = quandl.json()

In [86]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
type(quandl)

dict

The JSON response contains data in a dictionary format as expected. Looks like the body of the response is contained within an all-encompassing key called *dataset_data* which is noticeable right at the top of the response. 

Within this key, there is some header information such as *limit*, *split ratio*, *start_date* etc, as keys. However, the actual numerical data is contained in a key called *data* which we will explore further. Also, it looks like the data spans from 2012 to 2018. 

It is not apparent at first but the returned data is in fact very structured. The *column_names* key provides a list of the column names. In the *data* key, the values for these columns are stored. 

The *frequency* key tells the frequency of the data. 

In [22]:
data = quandl['dataset_data']['data']
print ("Number of data points: "+str(len(data)))

Number of data points: 1472


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

### Question 1

Collecting the data for AFX_X using the Quandl API

In [49]:
#Setting the URL with parameters for the request from the Quandl API and doing a request
url = "https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?&start_date=2017-01-01&end_date=2017-12-31&api_key="+API_KEY
response = requests.get(url)
response

<Response [200]>

### Question 2

Converting the JSON response to a Python dictionary

In [50]:
response = response.json()
type(response)

dict

### Question 3

Highest and lowest opening prices during the period

In [53]:
#Extracting the data key
data = response['dataset']['data']
print ("Number of trading days: "+str(len(data)))

Number of trading days: 255


In [60]:
#Finding the max and min opening prices by looping through daily responses
max_open_price = -1
min_open_price = 10000

for i in data:
    try:
        if i[1] > max_open_price:
            max_open_price = i[1]
        if i[1] < min_open_price:
            min_open_price = i[1]
    except: 
        continue
        
print ("Max open price: "+str(max_open_price))
print ("Min open price: "+str(min_open_price))

Max open price: 53.11
Min open price: 34.0


### Question 4

Highest day change between high and low price

In [63]:
#Finding the max daily price difference between high and low values
max_diff = -1

for i in data:
    try:
        if (i[2]-i[3]) > max_diff:
            max_diff = i[2]-i[3]
    except: 
        continue

print ("Highest day change between high and low prices: "+str(max_diff))

Highest day change between high and low prices: 2.8100000000000023


### Question 5

Largest change in closing price between any 2 days

In [87]:
#Finding the max change in closing price between 2 days
max_change_2_days = -1

for i in data:
    for j in data[1:]:
        try:
            if (abs(i[4]-j[4])) > max_change_2_days:
                max_change_2_days = abs(i[4]-j[4])
        except:
            continue

print ("Largest change in closing price between any 2 days: "+str(max_change_2_days))

Largest change in closing price between any 2 days: 19.03


### Question 6

Average daily trading volume

In [68]:
#Storing all volume values in a list and finding the average
volume = [x[6] for x in data]

avg_vol = sum(volume)/len(volume)

print ("Average daily trading volume: "+str(avg_vol))

Average daily trading volume: 89124.33725490196


### Question 7

Median daily trading volume

In [84]:
def median(x):
    x = sorted(x)
    if len(x)%2==0:
        return (x[round(len(x)/2)] + x[round((len(x)/2))-1]/2)
    else:
        return x[(round(len(x)/2))-1]

In [85]:
#Storing all volume values in a list and finding the median
volume = [x[6] for x in data]

med_vol = median(volume)

print ("Median daily trading volume: "+str(med_vol))

Median daily trading volume: 76286.0
